In [1]:
month = ["08", "09", "10"]
team = ["SK", "HH", "WO", "HT", "SS", "LT", "OB", "LG","KT","NC"]

In [65]:
def craw_attendanve(month, team):
    game_dic = []
    
    # 데이터 추출
    for m in month:
        for t in team:
            url = "https://www.koreabaseball.com/ws/Schedule.asmx/GetScheduleList?leId=1&srIdList=0%2C9&seasonId=2009&gameMonth={}&teamId={}".format(m, t)
            response = requests.get(url)
            data = response.json()["rows"]
            
            for n in range(0,31):
                try:
                    date = data[n]['row'][0]['Text'] #날짜
                    time = data[n]['row'][1]['Text'] #시간
                    result = data[n]['row'][2]['Text'] #경기결과
                    park = data[n]['row'][7]['Text']  #구장
                    etc = data[n]['row'][8]['Text'] #비고
                            
                    game_dic.append({
                        "dates" : date,
                        "times" : time,
                        "results" : result,
                        "parks" : park,
                        "etcs" : etc,            
                        })
                
                except:
                    break
                    
    # 데이터 전처리
    game = pd.DataFrame(game_dic)
    game['times'] = game['times'].str.replace('<b>',' ').str.replace('</b>',' ')
    game['results'] = game['results'].str.replace('</span><span>',' ').str.replace('</span></em><span>',' ').str.replace('<span>','').str.replace('</span>','').str.replace('<em>','').str.replace('</em>','').str.replace('<span class="win">',' win ').str.replace('<span class="lose">',' lose ')
    results_split = pd.DataFrame([x.split(' ') for x in sum([list([x]) for x in game["results"]], [])])
    dates_split = pd.DataFrame([x.split('(') for x in sum([list([x]) for x in game["dates"]], [])])
    game["away"] = results_split[0]
    game["homewin"] = results_split[4]
    game["home"] = results_split[6]
    game["dates"] = "2019." + dates_split[0]
    game["weekday"] = dates_split[1].str.replace(")","")
    game = game.drop(["results"],axis = 1)
    game = game[["dates", "parks", "away", "home", "homewin", "etcs", "times" ]]
    game['dates'] = game['dates'].str.replace(".","/")
    game = game.drop_duplicates(subset=['dates','parks'], keep='first')
    game = game.set_index('dates')
    return game
                    

In [66]:
game = craw_attendanve(month, team)

In [67]:
game.head(5)

,parks,away,home,homewin,etcs,times
dates,,,,,,
2019/08/01,잠실,SK,두산,win,-,17:00
2019/08/02,잠실,SK,두산,lose,-,17:00
2019/08/04,문학,히어로즈,SK,win,-,18:30
2019/08/05,문학,히어로즈,SK,win,-,18:30
2019/08/06,문학,히어로즈,SK,lose,-,18:30


In [68]:
game.to_csv('datas/kbo_2019_season_9-10.csv')